<p style = 'color:#E6E4E7 ; background-color:#00561B ; text-align:center ; font-size: 300%'> Kayak project - Get hotel data for Rouen

<p style = 'color:#00561B ; background-color:#E6E4E7 ; text-align:center ; font-size: 150%'> Project prepared by Stephanie Cotineau - #dsmpt-Paris-08

In [1]:
!pip install Scrapy -q

In [2]:
import os # For manipulate file 
import logging # For display logs
import scrapy # Import scrapy & scrapy.crawler
from scrapy.crawler import CrawlerProcess
import re

In [3]:
# Name of the file where the results will be saved
filename = "Rouen_multipage.json"

# If file already exists, delete it before crawling (because Scrapy will 
# concatenate the last and new results otherwise)
if filename in os.listdir('results_hotels/'):
        os.remove('results_hotels/' + filename)

# Declare a new CrawlerProcess with some settings
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.88 Safari/537.36'
process = CrawlerProcess(settings = {
    'USER_AGENT': user_agent,
    'LOG_LEVEL': logging.ERROR,
    "FEEDS": {
        'results_hotels/' + filename : {"format": "json"},
    }
})

In [4]:
from bs4 import BeautifulSoup 
import requests

In [5]:
# Rouen - 5 first pages
offset = 0
link_offset = []
while offset != 125:
    url = f"https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4As3zj5MGwAIB0gIkNTU0MmM0YmQtZTc0Mi00ZTcwLTgzY2QtYjhlOTZkNWJiODUy2AIE4AIB&sid=318bebf61bd36e4cdf8b55a0d0f1f93c&aid=304142&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4As3zj5MGwAIB0gIkNTU0MmM0YmQtZTc0Mi00ZTcwLTgzY2QtYjhlOTZkNWJiODUy2AIE4AIB%26sid%3D318bebf61bd36e4cdf8b55a0d0f1f93c%26sb_price_type%3Dtotal%26%26&ss=rouen&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=rouen&search_pageview_id=75cf5c27f0ed02c9&offset={offset}"
    #print(url)
    offset = offset + 25
    link_offset.append(url)

In [6]:
link_offset

['https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4As3zj5MGwAIB0gIkNTU0MmM0YmQtZTc0Mi00ZTcwLTgzY2QtYjhlOTZkNWJiODUy2AIE4AIB&sid=318bebf61bd36e4cdf8b55a0d0f1f93c&aid=304142&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4As3zj5MGwAIB0gIkNTU0MmM0YmQtZTc0Mi00ZTcwLTgzY2QtYjhlOTZkNWJiODUy2AIE4AIB%26sid%3D318bebf61bd36e4cdf8b55a0d0f1f93c%26sb_price_type%3Dtotal%26%26&ss=rouen&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=rouen&search_pageview_id=75cf5c27f0ed02c9&offset=0',
 'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4As3zj5MGwAIB0gIkNTU0MmM0YmQtZTc0Mi00ZTcwLTgzY2QtYjhlOTZkNWJiODUy2AIE4AIB&sid=318bebf61bd36e4cdf8b55a0d0f1f93c&aid=304142&

In [7]:
class RouenSpider(scrapy.Spider) :
    # Name of spider
    name = 'Rouen'
    
    # Starting URL
    start_urls = ['https://www.booking.com/']
    
    def start_requests(self):
        for d in ['https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4As3zj5MGwAIB0gIkNTU0MmM0YmQtZTc0Mi00ZTcwLTgzY2QtYjhlOTZkNWJiODUy2AIE4AIB&sid=318bebf61bd36e4cdf8b55a0d0f1f93c&aid=304142&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4As3zj5MGwAIB0gIkNTU0MmM0YmQtZTc0Mi00ZTcwLTgzY2QtYjhlOTZkNWJiODUy2AIE4AIB%26sid%3D318bebf61bd36e4cdf8b55a0d0f1f93c%26sb_price_type%3Dtotal%26%26&ss=rouen&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=rouen&search_pageview_id=75cf5c27f0ed02c9&offset=0',
                  'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4As3zj5MGwAIB0gIkNTU0MmM0YmQtZTc0Mi00ZTcwLTgzY2QtYjhlOTZkNWJiODUy2AIE4AIB&sid=318bebf61bd36e4cdf8b55a0d0f1f93c&aid=304142&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4As3zj5MGwAIB0gIkNTU0MmM0YmQtZTc0Mi00ZTcwLTgzY2QtYjhlOTZkNWJiODUy2AIE4AIB%26sid%3D318bebf61bd36e4cdf8b55a0d0f1f93c%26sb_price_type%3Dtotal%26%26&ss=rouen&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=rouen&search_pageview_id=75cf5c27f0ed02c9&offset=25',
                  'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4As3zj5MGwAIB0gIkNTU0MmM0YmQtZTc0Mi00ZTcwLTgzY2QtYjhlOTZkNWJiODUy2AIE4AIB&sid=318bebf61bd36e4cdf8b55a0d0f1f93c&aid=304142&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4As3zj5MGwAIB0gIkNTU0MmM0YmQtZTc0Mi00ZTcwLTgzY2QtYjhlOTZkNWJiODUy2AIE4AIB%26sid%3D318bebf61bd36e4cdf8b55a0d0f1f93c%26sb_price_type%3Dtotal%26%26&ss=rouen&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=rouen&search_pageview_id=75cf5c27f0ed02c9&offset=50',
                  'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4As3zj5MGwAIB0gIkNTU0MmM0YmQtZTc0Mi00ZTcwLTgzY2QtYjhlOTZkNWJiODUy2AIE4AIB&sid=318bebf61bd36e4cdf8b55a0d0f1f93c&aid=304142&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4As3zj5MGwAIB0gIkNTU0MmM0YmQtZTc0Mi00ZTcwLTgzY2QtYjhlOTZkNWJiODUy2AIE4AIB%26sid%3D318bebf61bd36e4cdf8b55a0d0f1f93c%26sb_price_type%3Dtotal%26%26&ss=rouen&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=rouen&search_pageview_id=75cf5c27f0ed02c9&offset=75',
                  'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4As3zj5MGwAIB0gIkNTU0MmM0YmQtZTc0Mi00ZTcwLTgzY2QtYjhlOTZkNWJiODUy2AIE4AIB&sid=318bebf61bd36e4cdf8b55a0d0f1f93c&aid=304142&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4As3zj5MGwAIB0gIkNTU0MmM0YmQtZTc0Mi00ZTcwLTgzY2QtYjhlOTZkNWJiODUy2AIE4AIB%26sid%3D318bebf61bd36e4cdf8b55a0d0f1f93c%26sb_price_type%3Dtotal%26%26&ss=rouen&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=rouen&search_pageview_id=75cf5c27f0ed02c9&offset=100']:
            yield scrapy.Request(d,callback=self.search)
    
    # Callback used after search launched
    def search(self, response):
        hotels = response.css('div.a826ba81c4.fe821aea6c.fa2f36ad22.afd256fc79.d08f526e0d.ed11e24d01.da89aeb942')
        for hotel in hotels:
            yield {
                'location' : hotel.css('span.f4bd0794db.b4273d69aa::text').get(),
                'hotel_url' : hotel.css('a.e13098a59f').attrib['href'],
                'hotel_name' : hotel.css('div.fcab3ed991.a23c043802::text').get(),
                'hotel_rating' : hotel.css('div.b5cd09854e.d10a6220b4::text').get(),
                'hotel_description_short' : hotel.css('div.d8eab2cf7f::text').get()
            }

In [8]:
# Start the crawling using the spider defined above
process.crawl(RouenSpider)
process.start()

In [9]:
import pandas as pd

In [10]:
rouen = pd.read_json('results_hotels/Rouen_multipage.json')

In [11]:
len(rouen)

125

In [12]:
rouen.head(55)

,location,hotel_url,hotel_name,hotel_rating,hotel_description_short
0,"Centre-ville de Rouen, Rouen",https://www.booking.com/hotel/fr/rouen-hyper-c...,Rouen - Centre Historique - Tout Confort et au...,"6,9",Hébergement géré par un particulier
1,Rouen,https://www.booking.com/hotel/fr/le-carre-roue...,Le Carré Rouennais - Appart'Hôtel Rouen Gare,"8,8",Hébergement géré par un particulier
2,"Centre-ville de Rouen, Rouen",https://www.booking.com/hotel/fr/coeur-de-roue...,Coeur de Rouen,"8,5","Situé en plein cœur de Rouen, à environ 1,4 km..."
3,"Centre-ville de Rouen, Rouen",https://www.booking.com/hotel/fr/cosyappart-le...,CosyAppart - LE SAINT OUEN,"8,5","Situé dans le centre de Rouen, à moins de 600 ..."
4,Rouen,https://www.booking.com/hotel/fr/residence-st-...,Residence Neoresid St Sever,"7,5","Dotée d'un salon commun, la Residence Neoresid..."
5,Rouen,https://www.booking.com/hotel/fr/appart-ty-bou...,Cosy'Appart - TY BOUQUET,"8,2",Le Cosy'Appart - TY BOUQUET est situé à Rouen....
6,"Centre-ville de Rouen, Rouen",https://www.booking.com/hotel/fr/appart-coeur-...,Appart Coeur de Rouen - Place Saint-Maclou,"9,2","Situé dans le centre de Rouen, à seulement 300..."
7,Rouen,https://www.booking.com/hotel/fr/cmg-rouen-riv...,CMG - Rouen Rive Droite - Gare,"8,3",Le CMG - Rouen Rive Droite - Gare propose des ...
8,"Centre-ville de Rouen, Rouen",https://www.booking.com/hotel/fr/l-39-espace-p...,L'Espace Privé Rouen,"9,5",L'espace Prive Rouen propose un studio avec ba...
9,"Centre-ville de Rouen, Rouen",https://www.booking.com/hotel/fr/rouenappartho...,Appart Rouen Centre,"8,1",Hébergement géré par un particulier


In [13]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.0) AppleWebKit/535.1 (KHTML, like Gecko) Chrome/13.0.782.112 Safari/535.1'}
result_ll =[]
for link in rouen['hotel_url']:
    page = requests.get(link, headers = headers)
    soup_link = BeautifulSoup(page.content, "lxml")
    lat_lon = soup_link.find_all(id = "hotel_address")
    for ll in lat_lon : 
        #print(ll['data-atlas-latlng'])
        result_ll.append(ll['data-atlas-latlng'])

In [14]:
len(result_ll)

125

In [15]:
rouen['Latitude_Longitude'] = result_ll

In [16]:
lat = []
lon = []

# For each row in a varible,
for row in rouen['Latitude_Longitude']:
    # Try to,
    try:
        # Split the row by comma and append
        # everything before the comma to lat
        lat.append(row.split(',')[0])
        # Split the row by comma and append
        # everything after the comma to lon
        lon.append(row.split(',')[1])
    # But if you get an error
    except:
        # append a missing value to lat
        lat.append(np.NaN)
        # append a missing value to lon
        lon.append(np.NaN)

# Create two new columns from lat and lon
rouen['latitude'] = lat
rouen['longitude'] = lon

In [17]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.0) AppleWebKit/535.1 (KHTML, like Gecko) Chrome/13.0.782.112 Safari/535.1'}
result_long_description =[]
for link in rouen['hotel_url']:
    page = requests.get(link, headers = headers)
    soup_link = BeautifulSoup(page.content, "lxml")
    long_description = soup_link.find_all(id = "property_description_content") 
    for ld in long_description : 
        result_long_description.append(ld.text)

In [18]:
len(result_long_description)

125

In [19]:
rouen['hotel_description_long'] = result_long_description

In [20]:
rouen.head(55)

,location,hotel_url,hotel_name,hotel_rating,hotel_description_short,Latitude_Longitude,latitude,longitude,hotel_description_long
0,"Centre-ville de Rouen, Rouen",https://www.booking.com/hotel/fr/rouen-hyper-c...,Rouen - Centre Historique - Tout Confort et au...,"6,9",Hébergement géré par un particulier,"49.44481780,1.08704270",49.44481780,1.08704270,\nVous pouvez bénéficier d'une réduction Geniu...
1,Rouen,https://www.booking.com/hotel/fr/le-carre-roue...,Le Carré Rouennais - Appart'Hôtel Rouen Gare,"8,8",Hébergement géré par un particulier,"49.45137780,1.09229250",49.45137780,1.09229250,\nLe Carré Rouennais - Appart'Hôtel Rouen Gare...
2,"Centre-ville de Rouen, Rouen",https://www.booking.com/hotel/fr/coeur-de-roue...,Coeur de Rouen,"8,5","Situé en plein cœur de Rouen, à environ 1,4 km...","49.44325675,1.10441055",49.44325675,1.10441055,\nVous pouvez bénéficier d'une réduction Geniu...
3,"Centre-ville de Rouen, Rouen",https://www.booking.com/hotel/fr/cosyappart-le...,CosyAppart - LE SAINT OUEN,"8,5","Situé dans le centre de Rouen, à moins de 600 ...","49.44369966,1.10166196",49.44369966,1.10166196,\nVous pouvez bénéficier d'une réduction Geniu...
4,Rouen,https://www.booking.com/hotel/fr/residence-st-...,Residence Neoresid St Sever,"7,5","Dotée d'un salon commun, la Residence Neoresid...","49.43001960,1.07948630",49.43001960,1.07948630,"\nDotée d'un salon commun, la Residence Neores..."
5,Rouen,https://www.booking.com/hotel/fr/appart-ty-bou...,Cosy'Appart - TY BOUQUET,"8,2",Le Cosy'Appart - TY BOUQUET est situé à Rouen....,"49.45028530,1.08908980",49.45028530,1.08908980,\nVous pouvez bénéficier d'une réduction Geniu...
6,"Centre-ville de Rouen, Rouen",https://www.booking.com/hotel/fr/appart-coeur-...,Appart Coeur de Rouen - Place Saint-Maclou,"9,2","Situé dans le centre de Rouen, à seulement 300...","49.44000310,1.09862010",49.44000310,1.09862010,"\nSitué dans le centre de Rouen, à seulement 3..."
7,Rouen,https://www.booking.com/hotel/fr/cmg-rouen-riv...,CMG - Rouen Rive Droite - Gare,"8,3",Le CMG - Rouen Rive Droite - Gare propose des ...,"49.44812080,1.09537360",49.44812080,1.09537360,\nLe CMG - Rouen Rive Droite - Gare propose de...
8,"Centre-ville de Rouen, Rouen",https://www.booking.com/hotel/fr/l-39-espace-p...,L'Espace Privé Rouen,"9,5",L'espace Prive Rouen propose un studio avec ba...,"49.44488500,1.08819185",49.44488500,1.08819185,\nL'espace Prive Rouen propose un studio avec ...
9,"Centre-ville de Rouen, Rouen",https://www.booking.com/hotel/fr/rouenappartho...,Appart Rouen Centre,"8,1",Hébergement géré par un particulier,"49.44113660,1.09534490",49.44113660,1.09534490,"\nSitué dans le centre de Rouen, à 700 mètres ..."


In [21]:
rouen['hotel_description_long'] = rouen['hotel_description_long'].replace('\n','', regex=True)

In [22]:
rouen.head(5)

,location,hotel_url,hotel_name,hotel_rating,hotel_description_short,Latitude_Longitude,latitude,longitude,hotel_description_long
0,"Centre-ville de Rouen, Rouen",https://www.booking.com/hotel/fr/rouen-hyper-c...,Rouen - Centre Historique - Tout Confort et au...,"6,9",Hébergement géré par un particulier,"49.44481780,1.08704270",49.44481780,1.08704270,Vous pouvez bénéficier d'une réduction Genius ...
1,Rouen,https://www.booking.com/hotel/fr/le-carre-roue...,Le Carré Rouennais - Appart'Hôtel Rouen Gare,"8,8",Hébergement géré par un particulier,"49.45137780,1.09229250",49.45137780,1.09229250,Le Carré Rouennais - Appart'Hôtel Rouen Gare p...
2,"Centre-ville de Rouen, Rouen",https://www.booking.com/hotel/fr/coeur-de-roue...,Coeur de Rouen,"8,5","Situé en plein cœur de Rouen, à environ 1,4 km...","49.44325675,1.10441055",49.44325675,1.10441055,Vous pouvez bénéficier d'une réduction Genius ...
3,"Centre-ville de Rouen, Rouen",https://www.booking.com/hotel/fr/cosyappart-le...,CosyAppart - LE SAINT OUEN,"8,5","Situé dans le centre de Rouen, à moins de 600 ...","49.44369966,1.10166196",49.44369966,1.10166196,Vous pouvez bénéficier d'une réduction Genius ...
4,Rouen,https://www.booking.com/hotel/fr/residence-st-...,Residence Neoresid St Sever,"7,5","Dotée d'un salon commun, la Residence Neoresid...","49.43001960,1.07948630",49.43001960,1.07948630,"Dotée d'un salon commun, la Residence Neoresid..."


In [23]:
rouen['hotel_description_long'][55]

"Vous pouvez bénéficier d'une réduction Genius dans l'établissement Welk'Home Loft Rouen CHU - Coeur Historique\xa0! Connectez-vous pour économiser.Situé à Rouen, à 1,8 km de la cathédrale Notre-Dame, le Welk'Home Loft Rouen CHU - Coeur Historique propose des hébergements avec un salon commun, une connexion Wi-Fi gratuite et une cuisine commune. Il offre une vue sur le jardin.Cet appartement comprend une chambre, une télévision par câble à écran plat, un lave-linge et une salle de bains pourvue d'une douche. Sa cuisine est équipée d'un micro-ondes et d'un lave-vaisselle. Les serviettes et le linge de lit sont fournis.Vous pourrez faire du vélo dans les environs.L'aéroport de Beauvais-Tillé, le plus proche, est à 84 km. "

In [24]:
# Clean location column
rouen['location'] = ["Rouen" if value!="Rouen" else value for value in rouen["location"]]

In [25]:
rouen.head()

,location,hotel_url,hotel_name,hotel_rating,hotel_description_short,Latitude_Longitude,latitude,longitude,hotel_description_long
0,Rouen,https://www.booking.com/hotel/fr/rouen-hyper-c...,Rouen - Centre Historique - Tout Confort et au...,"6,9",Hébergement géré par un particulier,"49.44481780,1.08704270",49.44481780,1.08704270,Vous pouvez bénéficier d'une réduction Genius ...
1,Rouen,https://www.booking.com/hotel/fr/le-carre-roue...,Le Carré Rouennais - Appart'Hôtel Rouen Gare,"8,8",Hébergement géré par un particulier,"49.45137780,1.09229250",49.45137780,1.09229250,Le Carré Rouennais - Appart'Hôtel Rouen Gare p...
2,Rouen,https://www.booking.com/hotel/fr/coeur-de-roue...,Coeur de Rouen,"8,5","Situé en plein cœur de Rouen, à environ 1,4 km...","49.44325675,1.10441055",49.44325675,1.10441055,Vous pouvez bénéficier d'une réduction Genius ...
3,Rouen,https://www.booking.com/hotel/fr/cosyappart-le...,CosyAppart - LE SAINT OUEN,"8,5","Situé dans le centre de Rouen, à moins de 600 ...","49.44369966,1.10166196",49.44369966,1.10166196,Vous pouvez bénéficier d'une réduction Genius ...
4,Rouen,https://www.booking.com/hotel/fr/residence-st-...,Residence Neoresid St Sever,"7,5","Dotée d'un salon commun, la Residence Neoresid...","49.43001960,1.07948630",49.43001960,1.07948630,"Dotée d'un salon commun, la Residence Neoresid..."


In [26]:
rouen.to_csv("Rouen.csv")